In [1]:
df = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv")
df #ignore the warnings

NameError: name 'pd' is not defined

In [ ]:
#ndvi_cols= [col for col in df.columns if '_N' in col]
#ndvi_data= df[ndvi_cols]
#ndvi_data.isna().sum()
#ndvi_scaled=ndvi_data
#ndvi_data


In [ ]:

from sklearn.preprocessing import StandardScaler
from scipy.signal import savgol_filter


place_col = df.columns[0:3]
ndvi_cols = df.columns[3:]

ndvi_data = df[ndvi_cols]
ndvi_data

In [ ]:
scaler = StandardScaler()
ndvi_scaled = pd.DataFrame(scaler.fit_transform(ndvi_data), columns=ndvi_cols)

In [ ]:
#def denoise_row(row):
    # Fill NaNs temporarily with linear interpolation before filtering
    #row = row.interpolate(limit_direction='both')
    # Apply Savitzky-Golay filter (window size must be odd and < len(row))
   # try:
   #     return savgol_filter(row, window_length=5, polyorder=2)
#    except:
 #       return row  # fallback if too many NaNs or too short

#ndvi_denoised = ndvi_scaled.apply(denoise_row, axis=1, result_type='expand')
#ndvi_denoised.columns = ndvi_cols

In [ ]:
#ndvi_final = ndvi_denoised.interpolate(axis=1, limit_direction='both')
ndvi_final = ndvi_scaled.interpolate(axis=1, limit_direction='both')
#ndvi_final = ndvi_denoised.fillna(ndvi_denoised.mean())
ndvi_final

In [ ]:
#df_clean = pd.concat([df[[place_col]], ndvi_final], axis=1)
#df_clean

In [ ]:



# Interpolate each row across time
#ndvi_interpolated = ndvi_data.interpolate(axis=1, limit_direction='both')

# Reattach the place column
#df_clean = pd.concat([df[[place_col]], ndvi_interpolated], axis=1)



In [ ]:
#from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler(feature_range=(-1, 1))
#ndvi_scaled = pd.DataFrame(scaler.fit_transform(ndvi_data), columns=ndvi_cols)
#ndvi_scaled

In [ ]:
#from scipy.stats import zscore
#z_scores = np.abs(zscore(ndvi_scaled, nan_policy='omit'))
# Mark as NaN where z > 3 (outlier)
#ndvi_no_outliers = ndvi_scaled.mask(z_scores > 3)
#ndvi_no_outliers

In [ ]:
#ndvi_filled = ndvi_no_outliers.fillna(ndvi_no_outliers.median())
#ndvi_filled

In [ ]:
df[ndvi_cols]= ndvi_final
df


In [ ]:
#df.fillna(df.mean(numeric_only=True), inplace=True) #simple mean imputation [This part has a lot of scope for imporovement.]
#keep in mind that the data is inherently noisy and the test dataset is not.
#df.isnull().sum()

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# Drop ID column
df.drop(columns=['ID'], inplace=True)

# Encode class column (if it's categorical)
label_encoder = LabelEncoder()
df['class'] = label_encoder.fit_transform(df['class'])

# Split into features and target
X = df.drop(columns=['class'])
y = df['class']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# Fit multinomial logistic regression
model = LogisticRegression(
    multi_class='multinomial',
    solver='lbfgs',
    max_iter=10
)
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Classification report with all original class labels
print(classification_report(
    y_test,
    y_pred,
    labels=list(range(len(label_encoder.classes_))),
    target_names=label_encoder.classes_
))


In [ ]:
test_data = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv")
test_data.shape

In [ ]:
ID=test_data['ID']
test_data.drop(['ID'],axis=1,inplace=True)

In [ ]:
y_test = model.predict(test_data)

In [ ]:
y_test

In [ ]:
y_decoded = label_encoder.inverse_transform(y_test)
y_decoded

In [ ]:
result = pd.DataFrame({
    'ID': ID,
    'class': y_decoded
})

In [ ]:
result

In [ ]:
result.to_csv("submission.csv", index=False) #this file will appear under the output section of the right navbar. You need to submit this csv file